Import required libraries for training

In [12]:
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import keras
import os
from pathlib import Path
import shutil

Prepare created dataset

In [13]:
raw_dataset = pd.read_csv("dataset/data.csv")
size = raw_dataset["size"][0]
# path,kana,font,size,antialias
labels_map = json.load(open("dataset/mapping.json"))


def load_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_png(image, channels=1)
    image = tf.image.convert_image_dtype(image, tf.float32)
    return image


def load_image_label(path, label):
    return load_image(path), label


def load_label(label):
    return labels_map[label]


def create_dataset():
    dataset = tf.data.Dataset.from_tensor_slices(
        (raw_dataset["path"], raw_dataset["kana"].map(load_label))
    )
    dataset = dataset.map(load_image_label)
    return dataset


def create_model():
    model = keras.models.Sequential(
        [
            # grayscale images 24x24, labels len(labels_map)
            keras.layers.Conv2D(32, (3, 3), activation="relu", input_shape=(size, size, 1)),
            keras.layers.MaxPooling2D((2, 2)),
            keras.layers.Conv2D(64, (3, 3), activation="relu"),
            keras.layers.MaxPooling2D((2, 2)),
            keras.layers.Conv2D(64, (3, 3), activation="relu"),
            keras.layers.Flatten(),
            keras.layers.Dense(64, activation="relu"),
            keras.layers.Dense(len(labels_map), activation="softmax"),
        ]
    )
    model.compile(
        optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
    )
    return model


model = create_model()
dataset = create_dataset()

# dataset = dataset.shuffle(len(raw_dataset), seed=42)
images = np.array([image for image, label in dataset])
labels = np.array([label for image, label in dataset])
# train_size = int(len(raw_dataset) * 0.8)
# train_dataset = dataset.take(train_size)
# train_images = np.array([image for image, label in train_dataset])
# train_labels = np.array([label for image, label in train_dataset])
# test_dataset = dataset.skip(train_size)
# test_images = np.array([image for image, label in test_dataset])
# test_labels = np.array([label for image, label in test_dataset])

/home/fufsob/vuz/tpr/venv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-05-12 12:49:57.271218: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-05-12 12:50:10.608060: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Train the model

In [14]:
print(images.shape)
print(labels)

(95658, 64, 64, 1)
[  0   1   2 ... 146 147 148]


In [15]:
np.random.seed(420)
output_dir = Path("output")
if output_dir.exists():
    shutil.rmtree(output_dir, ignore_errors=True)
output_dir.mkdir(exist_ok=True)
model.fit(
    images,
    labels,
    epochs=40,
    callbacks=[
        keras.callbacks.TensorBoard(log_dir='output/logs'),
        keras.callbacks.ModelCheckpoint('output/model_{epoch}.weights.h5', save_weights_only=True),
        keras.callbacks.ModelCheckpoint('output/model.keras', save_best_only=True),
        keras.callbacks.EarlyStopping("val_loss", patience=3),
    ],
    validation_freq=1,
    validation_split=0.2,
    shuffle=True,
)


Epoch 1/40
2392/2392 ━━━━━━━━━━━━━━━━━━━━ 126s 52ms/step - accuracy: 0.6906 - loss: 1.3118 - val_accuracy: 0.8880 - val_loss: 0.4039
Epoch 2/40
2392/2392 ━━━━━━━━━━━━━━━━━━━━ 107s 45ms/step - accuracy: 0.9663 - loss: 0.0885 - val_accuracy: 0.9150 - val_loss: 0.2983
Epoch 3/40
2392/2392 ━━━━━━━━━━━━━━━━━━━━ 110s 46ms/step - accuracy: 0.9769 - loss: 0.0547 - val_accuracy: 0.9151 - val_loss: 0.3311
Epoch 4/40
2392/2392 ━━━━━━━━━━━━━━━━━━━━ 106s 44ms/step - accuracy: 0.9792 - loss: 0.0464 - val_accuracy: 0.9231 - val_loss: 0.3163
Epoch 5/40
2392/2392 ━━━━━━━━━━━━━━━━━━━━ 111s 46ms/step - accuracy: 0.9830 - loss: 0.0384 - val_accuracy: 0.9216 - val_loss: 0.3579


Predict the output

In [16]:
model: keras.models.Sequential = keras.models.load_model('output/model.keras')
model.evaluate(images, labels)

2990/2990 ━━━━━━━━━━━━━━━━━━━━ 27s 9ms/step - accuracy: 0.9735 - loss: 0.0642


[0.09987399727106094, 0.9655961990356445]